<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Chat Completion

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using Chat Completion  
* Decision Scenario: Recommendation of a decision-making framework for specific higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [2]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.20.0"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.SemanticKernel, 1.20.0

Using Azure OpenAI Service


### Step 2 - Execute Chat Completion with Prompt Execution Settings

> "If you can’t describe what you are doing as a process, you don’t know what you’re doing." 
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [3]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide a list of decision frameworks that can help improve the quality of decisions.
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 2500, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

Console.WriteLine(chatResult.Content);

Certainly! Here’s a list of decision frameworks that can help improve the quality of decisions:

1. **SWOT Analysis**:
   - Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or project.

2. **Decision Matrix**:
   - A grid that helps prioritize options based on specific criteria, allowing for a systematic comparison.

3. **Cost-Benefit Analysis**:
   - Compares the costs and benefits of different options to determine the most financially viable choice.

4. **Pareto Analysis (80/20 Rule)**:
   - Focuses on identifying the 20% of causes that lead to 80% of the problems, helping prioritize efforts.

5. **Multi-Criteria Decision Analysis (MCDA)**:
   - Evaluates multiple conflicting criteria in decision-making, often using weighted scoring.

6. **The Eisenhower Matrix**:
   - Categorizes tasks based on urgency and importance to prioritize actions effectively.

7. **OODA Loop (Observe, Orient, Decide, Act)**:
   - A decision-making process that emphasizes agi

### Step 3 - Execute a Semantic Kernel Chat Completion with Streaming

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and service. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [4]:
// Stream the chat completion response
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt))
{
    Console.Write(content);
}

Certainly! Here’s a list of decision frameworks that can enhance the quality of decision-making:

1. **SWOT Analysis**: Evaluates the Strengths, Weaknesses, Opportunities, and Threats related to a decision or project, helping to clarify the factors involved.

2. **SMART Criteria**: Ensures that objectives set in decision-making are Specific, Measurable, Achievable, Relevant, and Time-bound.

3. **Decision Matrix**: A tool that uses a table to evaluate and prioritize different options based on selected criteria, allowing for a structured comparison.

4. **Pros and Cons List**: A simple yet effective way to weigh the advantages and disadvantages of each option systematically.

5. **Cost-Benefit Analysis**: Quantifies the expected costs and benefits of different choices, facilitating a clearer understanding of the potential return on investment.

6. **Pareto Analysis (80/20 Rule)**: Focuses on identifying the 20% of causes that lead to 80% of the problems or results, helping prioritize is

### Step 4 - Execute a Semantic Kernel with Chat History

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt
* User Message - A message from the user
* Assistante Message - A message from the LLM

This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [5]:
var systemPrompt = """
You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";
// Simple prompt to list 5 decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide five decision frameworks that can help improve the quality of decisions.
""";

var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}
chatHistory.AddAssistantMessage(chatResponse);

Improving decision quality involves utilizing structured frameworks that enhance clarity, analysis, and evaluation. Here are five effective decision frameworks you can consider:

### 1. **SWOT Analysis**
   - **Description**: Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or strategy.
   - **Application**:
     - **Strengths**: What do you do well? What are your advantages?
     - **Weaknesses**: What areas could be improved?
     - **Opportunities**: What external factors can be leveraged?
     - **Threats**: What obstacles or risks do you face?
   - **Usefulness**: This framework helps in understanding both internal and external factors influencing decisions.

### 2. **Decision Matrix**
   - **Description**: A quantitative tool that evaluates multiple options against various criteria to aid in decision-making.
   - **Application**:
     - List options in rows and criteria in columns.
     - Score each option against the criteria, weighting them base

### Step 5 - Using Semantic Kernel Chat History in Conversation

With Semantic Kernel maintaining state with the ChatHistory object, a user can interact with the LLM in a natural way. Note the interaction below, where the user is asking which of the particular decision frameworks are best suited to purchase a car. Without the ChatHistory being passed along, the LLM would not have had the proper context to answer the question.

> "Price is what you pay. Value is what you get."  
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

In [6]:
// Simple prompt that asks a follow-up question about the purchase of a car
// Note: No reference is made to what previous frameworks were listed
var simpleDecisionPromptFollowup = """
Which of these 5 decision frameworks would work best for making a decision to purchase a car. 
Select only the single best one.
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowup);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

For the decision to purchase a car, **Cost-Benefit Analysis** would be the best framework to use.

### Rationale:
1. **Financial Component**: A car purchase involves significant financial investment, including purchase price, financing options, insurance, maintenance, and fuel costs. A cost-benefit analysis allows you to comprehensively assess these costs against the benefits of owning the vehicle, such as convenience, transportation needs, and potential resale value.

2. **Comparison of Options**: It enables you to compare various models and features against their costs. By quantifying both ongoing costs and potential savings (like improved fuel efficiency or lower maintenance costs of a specific model), you can make an informed decision.

3. **Long-term Perspective**: This framework helps you consider both the short-term expenses (e.g., purchase price, immediate fees) and long-term implications (e.g., depreciation, maintenance costs) of owning a car.

### Steps to Apply the Framework

> "Take time to deliberate, but when the time for action comes, stop thinking and go in."
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [7]:
var simpleDecisionPromptFollowupPartTwo = """
Which of these 5 decision frameworks listed above are used by the military intelligence community?
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupPartTwo);

var chatResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
{
    chatResponse += content;
    Console.Write(content);
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

The military intelligence community often employs various decision frameworks for analysis, strategic planning, and operational decisions. Among the five frameworks listed, the following two are particularly relevant:

### 1. **SWOT Analysis**
   - **Application**: Military planners use SWOT analysis to evaluate strengths, weaknesses, opportunities, and threats related to specific missions, operational strategies, or emerging technologies. This framework aids in assessing both internal capabilities and external environments, which is crucial for strategic decision-making.

### 2. **Cost-Benefit Analysis**
   - **Application**: Cost-benefit analysis is frequently used to determine the feasibility and resource allocation of military operations and intelligence initiatives. This framework helps to quantify the expected costs (financial, manpower, time) against the anticipated benefits (e.g., mission success, intelligence gain), informing decisions on resource deployment and investment.

#

### Step 6 - Semantic Kernel inspecting Chat History 

The Semantic Kernel Chat History is a transparent object that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

In [9]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistory.Count());

foreach (var message in chatHistory)
{
    Console.WriteLine($"# {message.Role}: {message.Content}");
}

Number of chat interactions: 7
# system: You are a decision intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
# user: Provide five decision frameworks that can help improve the quality of decisions.
# assistant: Improving decision quality involves utilizing structured frameworks that enhance clarity, analysis, and evaluation. Here are five effective decision frameworks you can consider:

### 1. **SWOT Analysis**
   - **Description**: Evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or strategy.
   - **Application**:
     - **Strengths**: What do you do well? What are your advantages?
     - **Weaknesses**: What areas could be improved?
     - **Opportunities**: What external factors can be leveraged?
     - **Threats**: What obstacles or risks do you face?
   - **Usefulness**: This framewo